<font color="firebrick" size="6"><b>키워드 평균 가격 구하기 </b></font>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import requests
import time
import os

In [2]:
os.chdir("../../../smartstore") # signaturehelper module 호출하기 위해 경로 설정

In [9]:
data = pd.read_csv("conditionedData/data/data_231206/2_sort/2_sort_requests.csv")
keywords = data["relative_keyword"]

In [10]:
data.describe()

,total_search,total_store,competitive_rate
count,23323.000000,23323.000000,23323.000000
mean,5142.768512,27830.896240,10.242089
std,5429.775115,33280.256198,16.334166
min,1010.000000,31.000000,0.001176
25%,1670.000000,2545.000000,0.697932
50%,2930.000000,12757.000000,3.361905
75%,6190.000000,43246.500000,12.139381
max,29940.000000,130831.000000,123.432673


In [5]:
pd.options.display.max_rows = None  
keywords

0                     종아리압박밴드
1                        시카크림
2                        마사지젤
3                       약산성샴푸
4                         샴푸바
5                        아기로션
6                        아기크림
7                        아기세제
8                   어린이집생일답례품
9                      결혼식답례품
10                     티니핑마스크
11                      덴탈마스크
12                       전담액상
13                  비발디파크리프트권
14                      결혼식화환
15                       유럽유심
16                      캠핑카렌트
17                       태국유심
18                  비발디파크스키강습
19                  오피신유니버셀불리
20                     면세점립스틱
21                        CPB
22                  꾸셀베이스오리지널
23                      면세점향수
24                  인천공항면세점향수
25                     르라보면세점
26                   불리목욕하는여인
27                    면세점향수추천
28                      면세점시계
29                      밍크엘레팡
30                      콜드프레임
31                    다미아니귀걸이
32                      롱샴면세점
33        

**이상치 처리**

In [6]:
# 사분위수
def get_outlier(price_arr, weight=1.5):
  # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(price_arr, 25)
    quantile_75 = np.percentile(price_arr, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
  
    excluded_prices = price_arr[(price_arr < lowest) | (price_arr > highest)]
    price_arr = price_arr[(price_arr >=lowest) & (price_arr <= highest)]
    
    return price_arr,excluded_prices

# Z-score
def outlier_remove_z(data, threshold=3):
    z_scores = np.abs(data - np.mean(data)) / np.std(data) # Z-score 계산
    outlier = data[z_scores>threshold]
    removed_data = data[z_scores < threshold]
    
    return removed_data, outlier

In [7]:
# tmp = pd.DataFrame(columns=["relative_keyword", "avg_price"])
# tmp.to_csv("conditionedData/data/data_231206/3_avg_price/tmp.csv",index=False)

In [7]:
import requests
import json

avg_price = pd.read_csv("conditionedData/data/data_231206/3_avg_price/tmp.csv")
num = 19957                                                                                     

for keyword in keywords[num:]:
    cookies = {
        'NNB': '6TTK2RJU3JPWG',
        'autocomplete': 'use',
        'AD_SHP_BID': '23',
        '_ga': 'GA1.2.1198469368.1669035888',
        'NV_WETR_LOCATION_RGN_M': '"MDUxMTAxMTE="',
        'ASID': '6f5bbfa400000184fa368e5f00000067',
        'SHP_BUCKET_ID': '4',
        'NV_WETR_LAST_ACCESS_RGN_M': '"MDUxMTAxMTE="',
        'NID_AUT': 'xDAIttoc1Atdn7rAs4Y+rogecRjh3p1xevXbJnm0Nr0CEioRQ5/hhmRFwExAG7+T',
        'NID_JKL': 'b+0nlz0gwdbCoxClpc3utji+0en89OQHe0BEDbaUv/Y=',
        'nx_ssl': '2',
        'Viking_Rise_visit_time': '1695077035991',
        'page_uid': 'ifYbMdprvTVsse385PlssssssE0-160453',
        'spage_uid': 'ifYbMdprvTVsse385PlssssssE0-160453',
        'NID_SES': 'AAABxneIb+UGNfkjN7HA2gA5OlFQfYJVR367v6fonQYo1gBA+STpwnmSQKvPHCtKl9cSOobgAMZTANOk4CCrBDXAZ+8+xr6VQ9Ezi+8ApV32H/Cwi7PkDkvMuZ3QqwE0RmlD2gBOkdM8F5e33pm7MmNSfpFFusRSCaFir0+c2j3yksaJFDdG9PWn6RxWHTn0JXWu880PKz+pLfdysAibvruAtoxZbtdsvNi3dzUkiI74c14ilruNwmeJ+WmNLRwD79YtsmBgjQd8yEFd2P1MbddwtdfBqqrvIg9mgVAbGhHj86VEjcOAN0a8W84w7vJnGqR5AvPZp6ANSE7LBBYR6BN4RjlHsqne/i39lWovcl8zlv3X//TRlaZFjsdpIdPtHtEPum7wMp0eB+7nLpwAd3Wqz3Ku7eVW9ZH+cwnQwPuLEr3cfZT1scFz3Evmwa4DZWtNXVbACznRMQbUJMYBaN6+kt/PjN932m5O5S1ked50/18fduz9Ga5UzgUJwG86kA2Yj6uX0EJtos8uyYEuUpkHRgKtLkWwSDzI/Qq1281rtR5EslmrYPIicx0eGzIfCGV22q72LUyRwqROGp75LdZlMcC6mJd95X42HzhYacCFvSyx',
        'sus_val': 'oosk3W1yUa/jXC7KKHHcV8KC',
        'ncpa': '5114254|lnq2oiag|6088f8339ff87318d5466310aa33bd3df5f4ee8e|s_21a15037d5896|4a037b7e1ad4f4b2b46b7552bf557d83d6e09c69:4801924|lnq2r5yw|252c45b5b1c89d88d92fb4fd42bf99065652c3d4|s_1aec08c8356c9|4a0769694c3e60ffa7b878ff1e700b5ee7a31a93:95694|lnq2t46o|8f47110d725d91bc72e3083c0376c78965d77ffe|95694|28deee716df4986a1bc2f535eccb8e15165491ff:114|lnrg1u34|6c1e428c1fdcf6109a6e368afd36c2c6547842b5|s_1f0b6dd34481|0d9a8a153a4fd6f9e60247ee72a897423e0d11c7',
    }

    headers = {
        'authority': 'search.shopping.naver.com',
        'accept': 'application/json, text/plain, */*',
        'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        # 'cookie': 'NNB=6TTK2RJU3JPWG; autocomplete=use; AD_SHP_BID=23; _ga=GA1.2.1198469368.1669035888; NV_WETR_LOCATION_RGN_M="MDUxMTAxMTE="; ASID=6f5bbfa400000184fa368e5f00000067; SHP_BUCKET_ID=4; NV_WETR_LAST_ACCESS_RGN_M="MDUxMTAxMTE="; NID_AUT=xDAIttoc1Atdn7rAs4Y+rogecRjh3p1xevXbJnm0Nr0CEioRQ5/hhmRFwExAG7+T; NID_JKL=b+0nlz0gwdbCoxClpc3utji+0en89OQHe0BEDbaUv/Y=; nx_ssl=2; Viking_Rise_visit_time=1695077035991; page_uid=ifYbMdprvTVsse385PlssssssE0-160453; spage_uid=ifYbMdprvTVsse385PlssssssE0-160453; NID_SES=AAABxneIb+UGNfkjN7HA2gA5OlFQfYJVR367v6fonQYo1gBA+STpwnmSQKvPHCtKl9cSOobgAMZTANOk4CCrBDXAZ+8+xr6VQ9Ezi+8ApV32H/Cwi7PkDkvMuZ3QqwE0RmlD2gBOkdM8F5e33pm7MmNSfpFFusRSCaFir0+c2j3yksaJFDdG9PWn6RxWHTn0JXWu880PKz+pLfdysAibvruAtoxZbtdsvNi3dzUkiI74c14ilruNwmeJ+WmNLRwD79YtsmBgjQd8yEFd2P1MbddwtdfBqqrvIg9mgVAbGhHj86VEjcOAN0a8W84w7vJnGqR5AvPZp6ANSE7LBBYR6BN4RjlHsqne/i39lWovcl8zlv3X//TRlaZFjsdpIdPtHtEPum7wMp0eB+7nLpwAd3Wqz3Ku7eVW9ZH+cwnQwPuLEr3cfZT1scFz3Evmwa4DZWtNXVbACznRMQbUJMYBaN6+kt/PjN932m5O5S1ked50/18fduz9Ga5UzgUJwG86kA2Yj6uX0EJtos8uyYEuUpkHRgKtLkWwSDzI/Qq1281rtR5EslmrYPIicx0eGzIfCGV22q72LUyRwqROGp75LdZlMcC6mJd95X42HzhYacCFvSyx; sus_val=oosk3W1yUa/jXC7KKHHcV8KC; ncpa=5114254|lnq2oiag|6088f8339ff87318d5466310aa33bd3df5f4ee8e|s_21a15037d5896|4a037b7e1ad4f4b2b46b7552bf557d83d6e09c69:4801924|lnq2r5yw|252c45b5b1c89d88d92fb4fd42bf99065652c3d4|s_1aec08c8356c9|4a0769694c3e60ffa7b878ff1e700b5ee7a31a93:95694|lnq2t46o|8f47110d725d91bc72e3083c0376c78965d77ffe|95694|28deee716df4986a1bc2f535eccb8e15165491ff:114|lnrg1u34|6c1e428c1fdcf6109a6e368afd36c2c6547842b5|s_1f0b6dd34481|0d9a8a153a4fd6f9e60247ee72a897423e0d11c7',
        'logic': 'PART',
        'referer': 'https://search.shopping.naver.com/search/all?frm=NVSCPRO&origQuery&pagingSize=80&&productSet=total&query=%EC%84%B8%EC%A0%9C&sort=rel&timestamp=&viewType=list',
        'sbth': 'd222b49238bd2b7f29803e2ed2f08d01eedaa3d14d24417197e36287d8d5eae0ca4c01288a05847bfed167eb56487188',
        'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
        'sec-ch-ua-arch': '"x86"',
        'sec-ch-ua-bitness': '"64"',
        'sec-ch-ua-full-version-list': '"Chromium";v="118.0.5993.70", "Google Chrome";v="118.0.5993.70", "Not=A?Brand";v="99.0.0.0"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-model': '""',
        'sec-ch-ua-platform': '"Windows"',
        'sec-ch-ua-platform-version': '"15.0.0"',
        'sec-ch-ua-wow64': '?0',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
    }

    params = {
        'frm': 'NVSCPRO',
        'pagingIndex': '1',
        'pagingSize': '100',
        'productSet': 'total',
        'query': f"{keyword}",
        'sort': 'rel',
        'viewType': 'list',
    }
    
    time.sleep(np.random.rand()*1.3)

    try : 
        response = requests.get('https://search.shopping.naver.com/api/search/all', params=params, cookies=cookies, headers=headers)
        itemlist = json.loads(response.text)
        price_lst = []
        for idx,i in enumerate(itemlist['shoppingResult']['products']):
            price = i['price'],
            price =  int(price[0])
            price_lst.append(price)

        # 이상치 처리
        price_arr = np.array(price_lst)
        removed_arr,excluded_prices = get_outlier(price_arr, weight=1.5)
        print(f"{num}.{keyword}: {int(removed_arr.mean())} / {removed_arr.shape[0]} ")
        print(f"이상치 제거: {excluded_prices}\n")
        print(f"--------------------------------------------------------------------------------------\n")

        tmp = pd.DataFrame({"relative_keyword":keyword,"avg_price":[int(removed_arr.mean())]})    
        avg_price = pd.concat([avg_price,tmp],axis=0)
        
    except : 
        print(f"{num}.{keyword}")
        print(f"결측값\n")
        print(f"--------------------------------------------------------------------------------------\n")
        time.sleep(np.random.rand()*2)
        tmp = pd.DataFrame({"relative_keyword":[None],"avg_price":[None]})    
        avg_price = pd.concat([avg_price,tmp],axis=0)
        
    avg_price.to_csv("conditionedData/data/data_231206/3_avg_price//tmp.csv",index=False)    
    num += 1

19957.백일토퍼: 6845 / 90 
이상치 제거: [18000 22000 29800 20990 19500 29900 19900 32800 26800 19900]

--------------------------------------------------------------------------------------

19958.대두스탠딩: 18448 / 99 
이상치 제거: [77000]

--------------------------------------------------------------------------------------

19959.돌잡이용품: 11413 / 94 
이상치 제거: [37000 38900 39900 59300 37600 52900]

--------------------------------------------------------------------------------------

19960.우드토퍼: 22460 / 97 
이상치 제거: [51300 69000 69000]

--------------------------------------------------------------------------------------

19961.오광이벤트: 9190 / 74 
이상치 제거: [ 38300  38290  38290  46800 248200  43500  48300 106700 303000 410000
  70800  38290  45500]

--------------------------------------------------------------------------------------

19962.돌잡이: 10343 / 88 
이상치 제거: [ 56000  75000 116950  87000 199000  54000  64000  88000  89000  95000
 259000 164300]

-----------------------------------------------------

20006.칡즙: 33327 / 97 
이상치 제거: [105000  98900  79900]

--------------------------------------------------------------------------------------

20007.생칡즙: 33646 / 96 
이상치 제거: [ 69800 105000  87000  77500]

--------------------------------------------------------------------------------------

20008.토마토즙: 29709 / 93 
이상치 제거: [ 75000  59670 102610    100  56000  71000  56000]

--------------------------------------------------------------------------------------

20009.흑마늘즙: 41966 / 95 
이상치 제거: [169000 118000 118000 118000 664450]

--------------------------------------------------------------------------------------

20010.배도라지청: 27176 / 97 
이상치 제거: [ 96000  61200 111270]

--------------------------------------------------------------------------------------

20011.흑마늘진액: 44142 / 95 
이상치 제거: [169000 118000 163000 233000 664450]

--------------------------------------------------------------------------------------

20012.도라지즙: 25497 / 90 
이상치 제거: [ 58000  58900  74000  58000  69000 108600

20055.호주폴리코사놀: 129412 / 89 
이상치 제거: [ 500000 1100000  370000  750000  585000 1440000  785000  393000 1040000
  720000  364000]

--------------------------------------------------------------------------------------

20056.로얄케네디언: 150936 / 94 
이상치 제거: [600000 600000 450000 420000 450000 420000]

--------------------------------------------------------------------------------------

20057.사탕수수: 17869 / 99 
이상치 제거: [55750]

--------------------------------------------------------------------------------------

20058.MVL: 197768 / 94 
이상치 제거: [1100000  750000  750000  750000 1040000  678000]

--------------------------------------------------------------------------------------

20059.프롬바이오보스웰리아: 89258 / 100 
이상치 제거: []

--------------------------------------------------------------------------------------

20060.보스웰리아콘드로이친: 54636 / 92 
이상치 제거: [198000 450000 237600 300000 289800 255030 256150 225700]

--------------------------------------------------------------------------------------



20103.카테킨: 27946 / 97 
이상치 제거: [ 99900 179800  88000]

--------------------------------------------------------------------------------------

20104.녹차카테킨: 25196 / 95 
이상치 제거: [ 67790  75000 450000  79000 110000]

--------------------------------------------------------------------------------------

20105.메타그린부스터샷: 52445 / 99 
이상치 제거: [149900]

--------------------------------------------------------------------------------------

20106.맥스컷: 67385 / 97 
이상치 제거: [207000 400000 260000]

--------------------------------------------------------------------------------------

20107.메타그린골드: 66425 / 96 
이상치 제거: [250000 224930 249000 267300]

--------------------------------------------------------------------------------------

20108.맥스컷버닝: 56294 / 86 
이상치 제거: [400000 600000 200000 400000 600000 600000 200000 516000 344000 400000
 200000 600000 238000 474000]

--------------------------------------------------------------------------------------

20109.메타그린슬림: 55113 / 86 
이상치 제거: [353980 384

20152.지누스토퍼: 106900 / 97 
이상치 제거: [236690 243100 406000]

--------------------------------------------------------------------------------------

20153.슬로우토퍼: 136070 / 89 
이상치 제거: [439000 459000 549000 450000 429900 429900 527690 459000 447810 468000
 429900]

--------------------------------------------------------------------------------------

20154.누잠: 70466 / 26 
이상치 제거: []

--------------------------------------------------------------------------------------

20155.템퍼토퍼: 137781 / 92 
이상치 제거: [1425000 1980000 1248000 1154300 1154300 1227910 1092000 1710000]

--------------------------------------------------------------------------------------

20156.오시아지거리미티드: 767663 / 98 
이상치 제거: [ 452400 1154600]

--------------------------------------------------------------------------------------

20157.볼락루어대: 187624 / 97 
이상치 제거: [ 782000 1380000 1852400]

--------------------------------------------------------------------------------------

20158.에깅로드: 280562 / 96 
이상치 제거: [1755000  9660

...

23085.아마로네: 156318 / 85 
이상치 제거: [585300]

--------------------------------------------------------------------------------------

23086.나파밸리와인: 80609 / 87 
이상치 제거: [  922400   988200  4099900 19500000  3789900  3759900  3929900 18560900
  3799900 18530600  4009900 18568100 18515900]

--------------------------------------------------------------------------------------

23087.모스카토: 17356 / 92 
이상치 제거: [35000 34600 59900 38000 37800 42000 34900 89000]

--------------------------------------------------------------------------------------

23088.산토리가쿠빈: 30111 / 87 
이상치 제거: [ 95500 134900  99800 192900 102000 153000 114900 158500 219900  98900
 165000 206900 104310]

--------------------------------------------------------------------------------------

23089.조니워커블루: 89274 / 91 
이상치 제거: [ 434000 1485100  700650  802000  568780  751580  684700  741250  592700]

--------------------------------------------------------------------------------------

23090.로얄살루트: 47225 / 84 
이상치 제거: [269900 

23131.GB57820: 102324 / 99 
이상치 제거: [1327150]

--------------------------------------------------------------------------------------

23132.델코AGM70: 131901 / 100 
이상치 제거: []

--------------------------------------------------------------------------------------

23133.90L배터리: 71452 / 89 
이상치 제거: [149000 162200 205500 145610 143900 205450 144800 281600 144000 148500
 143900]

--------------------------------------------------------------------------------------

23134.델코80L: 70537 / 100 
이상치 제거: []

--------------------------------------------------------------------------------------

23135.불스원유막제거: 10934 / 86 
이상치 제거: [32980 40420 80840 36980 42300 53100 35260 35260 32680 34400 32680 38400
 34400 37840]

--------------------------------------------------------------------------------------

23136.자동차유막제거: 15801 / 98 
이상치 제거: [55000 59800]

--------------------------------------------------------------------------------------

23137.산화세륨: 16879 / 97 
이상치 제거: [ 65000 138160  68000]

--

23182.보쉬에어로트윈: 23302 / 78 
이상치 제거: [14750  6500 26000 30500 20000 26000 19000 14000 26070 19500 27000  9110
 20320 20000  7980 21020 27540 12310 30000 26850 25650 10200]

--------------------------------------------------------------------------------------

23183.불스원메탈실리콘와이퍼: 28151 / 98 
이상치 제거: [ 258500 1232500]

--------------------------------------------------------------------------------------

23184.모닝와이퍼: 9952 / 93 
이상치 제거: [29800 40900 40900 52800 44300 39800 52800]

--------------------------------------------------------------------------------------

23185.GV80와이퍼: 12477 / 96 
이상치 제거: [49200 61600 47300 41500]

--------------------------------------------------------------------------------------

23186.킴블레이드와이퍼: 23683 / 98 
이상치 제거: [   500 514345]

--------------------------------------------------------------------------------------

23187.바나나잎차: 16840 / 90 
이상치 제거: [ 41940  45900  51920  68000  68900  63600  46100 103950  57200  46900]

---------------------------------

23231.케이블타이체인: 15999 / 95 
이상치 제거: [70310 64770 88000 72000 72000]

--------------------------------------------------------------------------------------

23232.부산후쿠오카배: 560494 / 100 
이상치 제거: []

--------------------------------------------------------------------------------------

23233.후쿠오카배: 503496 / 99 
이상치 제거: [1595600]

--------------------------------------------------------------------------------------

23234.퀸비틀: 548887 / 100 
이상치 제거: []

--------------------------------------------------------------------------------------

23235.카멜리아호: 309094 / 92 
이상치 제거: [ 927100 1549900 1079900  679900  765900  839900  837100 1369900]

--------------------------------------------------------------------------------------

23236.보라카이픽업샌딩: 380572 / 100 
이상치 제거: []

--------------------------------------------------------------------------------------

23237.오사카배편: 803714 / 88 
이상치 제거: [  69000   85000  125000  125000  135000  135000 1549900 1479900 1579900
 1439900 1519900 1429900]

----

23284.패러글라이딩: 97615 / 94 
이상치 제거: [ 254500  250000  500000  264285 2722190 2651220]

--------------------------------------------------------------------------------------

23285.양평패러글라이딩: 170905 / 87 
이상치 제거: [1110000  700000  600500  464000  823000  534000  465000  724000  757000
  473000  426100  770000  508000]

--------------------------------------------------------------------------------------

23286.제주패러글라이딩: 357877 / 100 
이상치 제거: []

--------------------------------------------------------------------------------------

23287.제주도패러글라이딩: 357877 / 100 
이상치 제거: []

--------------------------------------------------------------------------------------

23288.단양패러글라이딩: 90257 / 71 
이상치 제거: [ 20000  60000  49000 160000  20000  63800  69750 120000 150000  60000
 155000 205000  60000 220000 170000 120000 150000 170000 150000 120000
 120000 120000 120000  72000  20000 135000 135000 135000 135000]

--------------------------------------------------------------------------------------

2

In [12]:
pd.options.display.max_rows = 100

In [13]:
avg_price = pd.read_csv("conditionedData/data/data_231206/3_avg_price/3_avg_price_only_request.csv")
avg_price

,relative_keyword,avg_price
0,종아리압박밴드,15833.0
1,시카크림,22415.0
2,마사지젤,23889.0
3,약산성샴푸,19151.0
4,샴푸바,12207.0
...,...,...
23318,나이키에코백,15848.0
23319,셀린느에코백,1828002.0
23320,이자벨마랑에코백,219997.0
23321,아페쎄에코백,97723.0


In [14]:
data

,relative_keyword,total_search,total_store,competitive_rate
0,종아리압박밴드,11230.0,80106.0,7.133215
1,시카크림,25760.0,85425.0,3.316188
2,마사지젤,5230.0,94873.0,18.140153
3,약산성샴푸,14540.0,124605.0,8.569807
4,샴푸바,16920.0,57113.0,3.375473
...,...,...,...,...
23318,나이키에코백,5590.0,1848.0,0.330590
23319,셀린느에코백,8900.0,176.0,0.019775
23320,이자벨마랑에코백,5450.0,2034.0,0.373211
23321,아페쎄에코백,9120.0,12553.0,1.376425


In [15]:
data = pd.merge(data,avg_price, on = "relative_keyword")

In [16]:
data

,relative_keyword,total_search,total_store,competitive_rate,avg_price
0,종아리압박밴드,11230.0,80106.0,7.133215,15833.0
1,시카크림,25760.0,85425.0,3.316188,22415.0
2,마사지젤,5230.0,94873.0,18.140153,23889.0
3,약산성샴푸,14540.0,124605.0,8.569807,19151.0
4,샴푸바,16920.0,57113.0,3.375473,12207.0
...,...,...,...,...,...
23317,나이키에코백,5590.0,1848.0,0.330590,15848.0
23318,셀린느에코백,8900.0,176.0,0.019775,1828002.0
23319,이자벨마랑에코백,5450.0,2034.0,0.373211,219997.0
23320,아페쎄에코백,9120.0,12553.0,1.376425,97723.0


In [19]:
data.to_csv("conditionedData/data/data_231206/3_avg_price/3_avg_price_request.csv",index=False)

In [ ]:
data

<font color="firebrick" size="6"><b> 키워드 별 평균 매출 </b></font>

In [ ]:
import requests
import json

keyword = "갤럭시탭S9필름"

cookies = {
    'NNB': '6TTK2RJU3JPWG',
    'autocomplete': 'use',
    'AD_SHP_BID': '23',
    '_ga': 'GA1.2.1198469368.1669035888',
    'NV_WETR_LOCATION_RGN_M': '"MDUxMTAxMTE="',
    'ASID': '6f5bbfa400000184fa368e5f00000067',
    'SHP_BUCKET_ID': '4',
    'NV_WETR_LAST_ACCESS_RGN_M': '"MDUxMTAxMTE="',
    'NID_AUT': 'xDAIttoc1Atdn7rAs4Y+rogecRjh3p1xevXbJnm0Nr0CEioRQ5/hhmRFwExAG7+T',
    'NID_JKL': 'b+0nlz0gwdbCoxClpc3utji+0en89OQHe0BEDbaUv/Y=',
    'nx_ssl': '2',
    'Viking_Rise_visit_time': '1695077035991',
    'page_uid': 'ifYbMdprvTVsse385PlssssssE0-160453',
    'spage_uid': 'ifYbMdprvTVsse385PlssssssE0-160453',
    'NID_SES': 'AAABxneIb+UGNfkjN7HA2gA5OlFQfYJVR367v6fonQYo1gBA+STpwnmSQKvPHCtKl9cSOobgAMZTANOk4CCrBDXAZ+8+xr6VQ9Ezi+8ApV32H/Cwi7PkDkvMuZ3QqwE0RmlD2gBOkdM8F5e33pm7MmNSfpFFusRSCaFir0+c2j3yksaJFDdG9PWn6RxWHTn0JXWu880PKz+pLfdysAibvruAtoxZbtdsvNi3dzUkiI74c14ilruNwmeJ+WmNLRwD79YtsmBgjQd8yEFd2P1MbddwtdfBqqrvIg9mgVAbGhHj86VEjcOAN0a8W84w7vJnGqR5AvPZp6ANSE7LBBYR6BN4RjlHsqne/i39lWovcl8zlv3X//TRlaZFjsdpIdPtHtEPum7wMp0eB+7nLpwAd3Wqz3Ku7eVW9ZH+cwnQwPuLEr3cfZT1scFz3Evmwa4DZWtNXVbACznRMQbUJMYBaN6+kt/PjN932m5O5S1ked50/18fduz9Ga5UzgUJwG86kA2Yj6uX0EJtos8uyYEuUpkHRgKtLkWwSDzI/Qq1281rtR5EslmrYPIicx0eGzIfCGV22q72LUyRwqROGp75LdZlMcC6mJd95X42HzhYacCFvSyx',
    'sus_val': 'oosk3W1yUa/jXC7KKHHcV8KC',
    'ncpa': '5114254|lnq2oiag|6088f8339ff87318d5466310aa33bd3df5f4ee8e|s_21a15037d5896|4a037b7e1ad4f4b2b46b7552bf557d83d6e09c69:4801924|lnq2r5yw|252c45b5b1c89d88d92fb4fd42bf99065652c3d4|s_1aec08c8356c9|4a0769694c3e60ffa7b878ff1e700b5ee7a31a93:95694|lnq2t46o|8f47110d725d91bc72e3083c0376c78965d77ffe|95694|28deee716df4986a1bc2f535eccb8e15165491ff:114|lnrg1u34|6c1e428c1fdcf6109a6e368afd36c2c6547842b5|s_1f0b6dd34481|0d9a8a153a4fd6f9e60247ee72a897423e0d11c7',
}

headers = {
    'authority': 'search.shopping.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    # 'cookie': 'NNB=6TTK2RJU3JPWG; autocomplete=use; AD_SHP_BID=23; _ga=GA1.2.1198469368.1669035888; NV_WETR_LOCATION_RGN_M="MDUxMTAxMTE="; ASID=6f5bbfa400000184fa368e5f00000067; SHP_BUCKET_ID=4; NV_WETR_LAST_ACCESS_RGN_M="MDUxMTAxMTE="; NID_AUT=xDAIttoc1Atdn7rAs4Y+rogecRjh3p1xevXbJnm0Nr0CEioRQ5/hhmRFwExAG7+T; NID_JKL=b+0nlz0gwdbCoxClpc3utji+0en89OQHe0BEDbaUv/Y=; nx_ssl=2; Viking_Rise_visit_time=1695077035991; page_uid=ifYbMdprvTVsse385PlssssssE0-160453; spage_uid=ifYbMdprvTVsse385PlssssssE0-160453; NID_SES=AAABxneIb+UGNfkjN7HA2gA5OlFQfYJVR367v6fonQYo1gBA+STpwnmSQKvPHCtKl9cSOobgAMZTANOk4CCrBDXAZ+8+xr6VQ9Ezi+8ApV32H/Cwi7PkDkvMuZ3QqwE0RmlD2gBOkdM8F5e33pm7MmNSfpFFusRSCaFir0+c2j3yksaJFDdG9PWn6RxWHTn0JXWu880PKz+pLfdysAibvruAtoxZbtdsvNi3dzUkiI74c14ilruNwmeJ+WmNLRwD79YtsmBgjQd8yEFd2P1MbddwtdfBqqrvIg9mgVAbGhHj86VEjcOAN0a8W84w7vJnGqR5AvPZp6ANSE7LBBYR6BN4RjlHsqne/i39lWovcl8zlv3X//TRlaZFjsdpIdPtHtEPum7wMp0eB+7nLpwAd3Wqz3Ku7eVW9ZH+cwnQwPuLEr3cfZT1scFz3Evmwa4DZWtNXVbACznRMQbUJMYBaN6+kt/PjN932m5O5S1ked50/18fduz9Ga5UzgUJwG86kA2Yj6uX0EJtos8uyYEuUpkHRgKtLkWwSDzI/Qq1281rtR5EslmrYPIicx0eGzIfCGV22q72LUyRwqROGp75LdZlMcC6mJd95X42HzhYacCFvSyx; sus_val=oosk3W1yUa/jXC7KKHHcV8KC; ncpa=5114254|lnq2oiag|6088f8339ff87318d5466310aa33bd3df5f4ee8e|s_21a15037d5896|4a037b7e1ad4f4b2b46b7552bf557d83d6e09c69:4801924|lnq2r5yw|252c45b5b1c89d88d92fb4fd42bf99065652c3d4|s_1aec08c8356c9|4a0769694c3e60ffa7b878ff1e700b5ee7a31a93:95694|lnq2t46o|8f47110d725d91bc72e3083c0376c78965d77ffe|95694|28deee716df4986a1bc2f535eccb8e15165491ff:114|lnrg1u34|6c1e428c1fdcf6109a6e368afd36c2c6547842b5|s_1f0b6dd34481|0d9a8a153a4fd6f9e60247ee72a897423e0d11c7',
    'logic': 'PART',
    'referer': 'https://search.shopping.naver.com/search/all?frm=NVSCPRO&origQuery&pagingSize=80&&productSet=total&query=%EC%84%B8%EC%A0%9C&sort=rel&timestamp=&viewType=list',
    'sbth': 'd222b49238bd2b7f29803e2ed2f08d01eedaa3d14d24417197e36287d8d5eae0ca4c01288a05847bfed167eb56487188',
    'sec-ch-ua': '"Chromium";v="118", "Google Chrome";v="118", "Not=A?Brand";v="99"',
    'sec-ch-ua-arch': '"x86"',
    'sec-ch-ua-bitness': '"64"',
    'sec-ch-ua-full-version-list': '"Chromium";v="118.0.5993.70", "Google Chrome";v="118.0.5993.70", "Not=A?Brand";v="99.0.0.0"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-model': '""',
    'sec-ch-ua-platform': '"Windows"',
    'sec-ch-ua-platform-version': '"15.0.0"',
    'sec-ch-ua-wow64': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36',
}

params = {
    'frm': 'NVSCPRO',
    'pagingIndex': '1',
    'pagingSize': '100',
    'productSet': 'total',
    'query': f"{keyword}",
    'sort': 'rel',
    'viewType': 'list',
}

response = requests.get('https://search.shopping.naver.com/api/search/all', params=params, cookies=cookies, headers=headers)
itemlist = json.loads(response.text)
itemlist
# sum = 0
# price_lst = []
# print(itemlist)
# # for idx,i in enumerate(itemlist['shoppingResult']['products']):
# #     price = i['price'],
# #     price =  int(price[0])
# #     price_lst.append(price)
    
# # price_arr = np.array(price_lst)
# # removed_arr,excluded_prices = get_outlier(price_arr, weight=1.5)
# # print(f"{keyword} \t {int(removed_arr.mean())} \t {removed_arr.shape[0]} ")
# # print(f"이상치 제거 {excluded_prices}\n")


In [ ]:
{'searchParam': {'sort': 'rel', 'pagingIndex': 1, 'pagingSize': '100', 'viewType': 'list', 'productSet': 'total', 'frm': 'NVSCPRO', 'query': '가습기오일', 'origQuery': '가습기오일', 'adQuery': '가습기오일', 'iq': [], 'eq': [], 'xq': [], 'timestamp': ''}, 'searchBanInfo': {'id': '62fb3e37c88eab787f1dafe6', 'keyword': '가습기용오일', 'handlingReason': 'SELL_STOP', 'banInfoType': 'BAN', 'serviceYn': 'Y', 'serviceEndDate': None, 'content1': '가습기용으로 광고/판매되는 오일/향료는 인체유해성 이슈로 관련 검색결과를 노출하지 않고 있으니 서비스 이용에 참고부탁드립니다.', 'content2': None, 'content3': None, 'linkContent': None, 'linkUrl': None, 'subkeywords': ['가습기용아로마오일', '가습기용아로마', '가습기용방향제', '아로마디퓨저가습기', '가습기에센셜', '가습기오일', '가습기아로마', '가습기아로마오일', '가습기방향제', '가습기용향료', '가습기향료', '가습기용에센셜']}, 'surls': [], 'nClick': {'code': 'svc.shopping.v3', 'codeType': 'ssc', 'pageUid': 'iTPJ1sqWc1U1nwtu5MC-277981', 'susVal': 'oosk3W1yUa/jXC7KKHHcV8KC', 'query': '%EA%B0%80%EC%8A%B5%EA%B8%B0%EC%98%A4%EC%9D%BC'}}

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10,6))
axs[0].boxplot(price_lst);
axs[1].hist(price_lst, bins=100);

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10,6))
axs[0].boxplot(removed_arr);
axs[1].hist(removed_arr, bins=100);


In [ ]:
def get_outlier(price_arr, weight=1.5):
  # target 값과 상관관계가 높은 열을 우선적으로 진행
    quantile_25 = np.percentile(price_arr, 25)
    quantile_75 = np.percentile(price_arr, 75)

    IQR = quantile_75 - quantile_25
    IQR_weight = IQR*weight
  
    lowest = quantile_25 - IQR_weight
    highest = quantile_75 + IQR_weight
  
    price_arr = price_arr[(price_arr >=lowest) & (price_arr <= highest)]
    return price_arr

In [ ]:
get_outlier(price_arr)

In [ ]:
plt.boxplot(get_outlier(price_arr));

In [ ]:
arr = price_arr

def remove_outliers(arr) :
    mean = arr.mean()
    std = arr.std()
    upper_bound = mean + (3 * std)
    lower_bound = mean - (3 * std)
    return (upper_bound,lower_bound)

upper_bound,lower_bound = remove_outliers(arr)

removed_arr = arr[(arr <=upper_bound) & (arr >= lower_bound)]
print(arr[(arr >=upper_bound) | (arr <= lower_bound)])
print(removed_arr.shape)
int(removed_arr.mean())